In [1]:
import sys
print(sys.executable)

/usr/local/opt/python@3.11/bin/python3.11


In [2]:
!pip install openai


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## Demo 1 - ChatBot with Assistant API

In [4]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo-1106",
)
show_json(assistant)

{'id': 'asst_wGQtVxga1nqkFMNnzv9IMRDb',
 'created_at': 1705807926,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [7]:
# 1. Create a thread
thread = client.beta.threads.create()
# 2. Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you help me to solve `3x + 1 = 10`, what's the value x?"
)
show_json(message)


{'id': 'msg_1Cgz8ONOihIzGQE8EkpL3lNw',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': "Can you help me to solve `3x + 1 = 10`, what's the value x?"},
   'type': 'text'}],
 'created_at': 1705808147,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_vPIC8U7fdaFsXRgSLXpf88Ir'}

In [8]:
# 3. Create a run
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
show_json(run)

{'id': 'run_J8lVF7YtDezQums1Qq2uiNsW',
 'assistant_id': 'asst_wGQtVxga1nqkFMNnzv9IMRDb',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1705808190,
 'expires_at': 1705808790,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_vPIC8U7fdaFsXRgSLXpf88Ir',
 'tools': [],
 'usage': None}

In [11]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [12]:
# 4. Retrieve the result from run
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_J8lVF7YtDezQums1Qq2uiNsW',
 'assistant_id': 'asst_wGQtVxga1nqkFMNnzv9IMRDb',
 'cancelled_at': None,
 'completed_at': 1705808191,
 'created_at': 1705808190,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1705808190,
 'status': 'completed',
 'thread_id': 'thread_vPIC8U7fdaFsXRgSLXpf88Ir',
 'tools': [],
 'usage': {'prompt_tokens': 49, 'completion_tokens': 6, 'total_tokens': 55}}

In [13]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_FHPZHWBElqD7LYgI4tmmzUq2',
   'assistant_id': 'asst_wGQtVxga1nqkFMNnzv9IMRDb',
   'content': [{'text': {'annotations': [], 'value': 'x = 3'},
     'type': 'text'}],
   'created_at': 1705808191,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_J8lVF7YtDezQums1Qq2uiNsW',
   'thread_id': 'thread_vPIC8U7fdaFsXRgSLXpf88Ir'},
  {'id': 'msg_1Cgz8ONOihIzGQE8EkpL3lNw',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': "Can you help me to solve `3x + 1 = 10`, what's the value x?"},
     'type': 'text'}],
   'created_at': 1705808147,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_vPIC8U7fdaFsXRgSLXpf88Ir'}],
 'object': 'list',
 'first_id': 'msg_FHPZHWBElqD7LYgI4tmmzUq2',
 'last_id': 'msg_1Cgz8ONOihIzGQE8EkpL3lNw',
 'has_more': False}

In [14]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_DgakHk4R8Tr5HWGP4X9uS2Rd',
   'assistant_id': 'asst_wGQtVxga1nqkFMNnzv9IMRDb',
   'content': [{'text': {'annotations': [],
      'value': 'Sure, you can solve for x by subtracting 1 from both sides of the equation to isolate the variable x.'},
     'type': 'text'}],
   'created_at': 1705808329,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_jTMW93EredZWGc4hGu7mSx0e',
   'thread_id': 'thread_vPIC8U7fdaFsXRgSLXpf88Ir'}],
 'object': 'list',
 'first_id': 'msg_DgakHk4R8Tr5HWGP4X9uS2Rd',
 'last_id': 'msg_DgakHk4R8Tr5HWGP4X9uS2Rd',
 'has_more': False}

## Demo 2  - Q&A document 

In [45]:
client = OpenAI()

assistant = client.beta.assistants.create(
    name="Finance Assistant",
    instructions="'You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview"
)

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [46]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run

In [47]:
# Upload the file
file = client.files.create(
    file=open(
        "data/language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)

# Update Assistant
assistant = client.beta.assistants.update(
    assistant.id,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id]
)
show_json(assistant)

{'id': 'asst_3Z6nq2PJXLOrJ4g8uZ83WxrG',
 'created_at': 1705827173,
 'description': None,
 'file_ids': ['file-YFuGSGvN5sN1YTciKOmtvBcx'],
 'instructions': "'You are a personal math tutor. Answer questions briefly, in a sentence or less.",
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Finance Assistant',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [48]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences."
)
run = wait_on_run(run, thread)
run.status

'completed'

In [49]:
# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


In [50]:
pretty_print(get_response(thread))

# Messages
user: What are some cool math concepts behind this ML paper pdf? Explain in two sentences.
assistant: It seems there was a problem with the content of the PDF you uploaded. The file might be corrupted or empty. Could you please check the file on your end and re-upload it if necessary?

